## Library

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import matplotlib.pyplot as plt
import tqdm
import wandb
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
import optuna
import regex
import json
from scipy.stats import hmean

## WanDB

In [ ]:
run = str(input('run 이름을 입력하세요 :'))
# selected_model = str(input('model 명을 입력하세요 (xgb/rf) :'))
opt = bool(input('Optuna 사용 여부를 입력하세요 (뭐라도 입력 시 사용) :'))

wandb.init(
    settings=wandb.Settings(start_method='thread'),
    dir=None,  # 로컬에 로그 저장하지 않음
    entity='remember-us', # team name,
    project='active', # project name
    name=run, # run name
)

## Data Load

In [ ]:
data_path: str = '/data/ephemeral/home/book/code/data/'
users = pd.read_csv(data_path + 'users.csv')
books = pd.read_csv(data_path + 'books.csv')
train = pd.read_csv(data_path + 'train_ratings.csv')
test = pd.read_csv(data_path + 'test_ratings.csv')
sub = pd.read_csv(data_path + 'sample_submission.csv')

## Data Preprocessing

In [ ]:
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    return x[2:-2].split(', ')

In [ ]:
def split_location(x: str) -> list:
    '''
    Parameters
    ----------
    x : str
        location 데이터

    Returns
    -------
    res : list
        location 데이터를 나눈 뒤, 정제한 결과를 반환합니다.
        순서는 country, state, city, ... 입니다.
    '''
    res = x.split(',')
    res = [i.strip().lower() for i in res]
    res = [regex.sub(r'[^a-zA-Z/ ]', '', i) for i in res]  # remove special characters
    res = [i if i not in ['n/a', ''] else np.nan for i in res]  # change 'n/a' into NaN
    res.reverse()  # reverse the list to get country, state, city, ... order

    for i in range(len(res)-1, 0, -1):
        if (res[i] in res[:i]) and (not pd.isna(res[i])):  # remove duplicated values if not NaN
            res.pop(i)

    return res

In [ ]:
def text_preprocessing(summary_text: str) -> str:
    '''
    주어진 텍스트 요약을 전처리합니다.

    1. 특수 문자 제거
    2. 알파벳과 숫자, 공백을 제외한 모든 문자 제거
    3. 여러 개의 공백을 하나의 공백으로
    4. 문자열의 앞뒤 공백 제거
    5. 모든 문자를 소문자로 변환

    Args:
        summary_text (str): 전처리할 텍스트 문자열

    Returns:
        str: 전처리된 텍스트 문자열. 입력이 NaN인 경우 'unknown' 반환.
    '''
    if pd.isna(summary_text):
        return 'unknown'  # NaN일 경우 'unknown' 반환
    
    summary_text = regex.sub('[.,\'\'''\'!?]', '', summary_text)  # 특수 문자 제거
    summary_text = regex.sub('[^0-9a-zA-Z\s]', '', summary_text)  # 알파벳과 숫자, 공백 제외한 문자 제거
    summary_text = regex.sub('\s+', ' ', summary_text)  # 여러 개의 공백을 하나의 공백으로
    summary_text = summary_text.lower()  # 소문자로 변환
    summary_text = summary_text.strip()  # 앞뒤 공백 제거
    return summary_text

In [ ]:
def categorize_publication(x: int, a: int) -> int:
    '''
    주어진 연도를 특정 기준에 따라 카테고리화하는 함수입니다.

    Parameters
    ----------
    x : int
        책의 발행 연도.
    a : int
        연도를 그룹화할 때 사용할 기준값 (예: 5년 단위로 그룹화).

    Returns
    -------
    int
        카테고리화된 연도를 반환합니다. 
        - 1970년 이하의 연도는 1970으로 반환합니다.
        - 2000년 초과의 연도는 2006으로 반환합니다.
        - 나머지 연도는 a 값에 맞게 그룹화하여 반환합니다.

    Example
    -------
    books['years'] = books['year_of_publication'].apply(lambda x: categorize_publication(x, 5))
    print(books['years'].value_counts())
    '''
    if x <= 1970:
        return 1970
    elif x > 2000:
        return 2006
    else:
        return x // a * a

In [ ]:
def extract_language_from_isbn(isbn):
    '''
    ISBN 정보를 사용하여 언어 코드를 추출하는 함수입니다.

    Parameters
    ----------
    isbn : str
        책의 ISBN 번호.

    Returns
    -------
    str
        ISBN에서 추출한 언어 코드. ISBN이 비어있거나 형식에 맞지 않을 경우 최빈값 'en'을 반환합니다.
        - isbn_language_map 참고
        - 기타 언어 코드: isbn_language_map에 정의된 국가 코드를 기반으로 반환
    '''
    isbn_language_map = {
        '0': 'en', '1': 'en', '2': 'fr', '3': 'de', '4': 'ja',
        '5': 'ru', '7': 'zh-CN', '82': 'no', '84': 'es', '87': 'da',
        '88': 'it', '89': 'ko', '94': 'nl', '600': 'fa', '602': 'ms',
        '606': 'ro', '604': 'vi', '618': 'el', '967': 'ms', '974': 'th',
        '989': 'pt'
    }
    if not isbn or not isbn.isdigit():
        return 'en'  # 기본값 영어권
    for prefix, language in isbn_language_map.items():
        if isbn.startswith(prefix):
            return language
    return 'en'  # 기본값 영어권

In [ ]:
def replace_language_using_isbn(books):
    '''
    ISBN 정보를 활용하여 language 결측치를 대체하는 함수입니다.

    Parameters
    ----------
    books : pd.DataFrame
        책 정보가 담긴 DataFrame. 반드시 'isbn' 및 'language' 열을 포함해야 합니다.

    Returns
    -------
    pd.DataFrame
        language 결측치가 ISBN 정보를 사용해 대체된 DataFrame. ISBN에서 언어를 추출할 수 없는 경우
        기본값 'en'으로 대체됩니다.

    Example
    -------
    books = replace_language_using_isbn(books)
    '''
    books['extracted_language'] = books['isbn'].apply(extract_language_from_isbn)
    books['language'] = books.apply(
        lambda row: row['extracted_language'] if pd.isna(row['language']) else row['language'],
        axis=1
    )
    books.drop(columns=['extracted_language'], inplace=True)
    return books

In [ ]:
def categorize_age(x: int, a: int) -> int:
    '''
    주어진 나이를 특정 기준에 따라 카테고리화하는 함수입니다.

    Parameters
    ----------
    x : int
        유저의 나이.
    a : int
        나이를 그룹화할 때 사용할 기준값 (예: 10년 단위로 그룹화).

    Returns
    -------
    int
        카테고리화된 나이를 반환합니다. 
        - 20년 미만의 나이는 10으로 반환합니다.
        - 60년 이상의 나이는 60으로 반환합니다.
        - 나머지 나이는 a 값에 맞게 그룹화하여 반환합니다.
    '''
    if x < 20:
        return 10
    elif x >= 60:
        return 60
    else:
        return x // a * a

In [ ]:
users_ = users.copy()
books_ = books.copy()

In [ ]:
# books 데이터 전처리
# book_title, book_author, publisher 열의 텍스트를 전처리
books_['book_title'] = books_['book_title'].apply(text_preprocessing)
books_['book_author'] = books_['book_author'].apply(text_preprocessing)
books_['publisher'] = books_['publisher'].apply(text_preprocessing)

# 발행 연도를 특정 기준으로 카테고리화하여 publication_range 열에 저
books_['publication_range'] = books_['year_of_publication'].apply(lambda x: categorize_publication(x, 5))

# ISBN 정보를 사용하여 결측된 language 열을 대체
books_ = replace_language_using_isbn(books_)

# category 열의 첫 번째 항목만 사용하며, 결측치가 있으면 NaN으로 설정
books_['category'] = books_['category'].apply(lambda x: str2list(x)[0] if not pd.isna(x) else np.nan)

# category 열의 텍스트를 전처리
books_['category'] = books_['category'].apply(text_preprocessing)

# 상위 카테고리 목록을 정의
high_categories = ['fiction', 'biography', 'history', 'religion', 'nonfiction', 'social', 'science', 'humor', 'body', 
                'business', 'economics', 'cook', 'health', 'fitness', 'famil', 'relationship', 
                'computer', 'travel', 'selfhelp', 'psychology', 'poetry', 'art', 'critic', 'nature', 'philosophy', 
                'reference','drama', 'sports', 'politic', 'comic', 'novel', 'craft', 'language', 'education', 'crime', 'music', 'pet', 
                'child', 'collection', 'mystery', 'garden', 'medical', 'author', 'house','technology', 'engineering', 'animal', 'photography',
                'adventure', 'game', 'science fiction', 'architecture', 'law', 'fantasy', 'antique', 'friend', 'brother', 'sister', 'cat',
                'math', 'christ', 'bible', 'fairy', 'horror', 'design', 'adolescence', 'actor', 'dog', 'transportation', 'murder', 'adultery', 'short', 'bear'
                ]

# high_category 열을 초기화
books_['high_category'] = None

# 각 카테고리에 대해 반복하며 매핑
for high_category in high_categories:
    # category 열에서 high_category가 포함된 행을 찾고, 해당 행의 high_category 열을 업데이트
    books_.loc[books_['category'].str.contains(high_category, case=False, na=False), 'high_category'] = high_category
books_['high_category'] = books_['high_category'].fillna('others') # 결측치를 'others'로 대체

# users 데이터 전처리
# age 열의 결측치를 평균값으로 대체
users_['age'] = users_['age'].fillna(users_['age'].mean())

# 나이를 특정 기준으로 카테고리화하여 age_range 열에 저장
users_['age_range'] = users_['age'].apply(lambda x: categorize_age(x, 10))

# location 데이터를 리스트로 분리하여 location_list 열에 저장
users_['location_list'] = users_['location'].apply(lambda x: split_location(x)) 

# location_list에서 첫 번째 요소를 location_country 열로, 두 번째 요소를 location_state 열로, 세 번째 요소를 location_city 열로 설정
users_['location_country'] = users_['location_list'].apply(lambda x: x[0])
users_['location_state'] = users_['location_list'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
users_['location_city'] = users_['location_list'].apply(lambda x: x[2] if len(x) > 2 else np.nan)

# 각 행을 반복하며 결측된 location_country나 location_state 값을 보완
for idx, row in users_.iterrows():
    if (not pd.isna(row['location_state'])) and pd.isna(row['location_country']):
        fill_country = users_[users_['location_state'] == row['location_state']]['location_country'].mode()
        fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
        users_.loc[idx, 'location_country'] = fill_country
    elif (not pd.isna(row['location_city'])) and pd.isna(row['location_state']):
        if not pd.isna(row['location_country']):
            fill_state = users_[(users_['location_country'] == row['location_country']) 
                                & (users_['location_city'] == row['location_city'])]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            users_.loc[idx, 'location_state'] = fill_state
        else:
            fill_state = users_[users_['location_city'] == row['location_city']]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            fill_country = users_[users_['location_city'] == row['location_city']]['location_country'].mode()
            fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
            users_.loc[idx, 'location_country'] = fill_country
            users_.loc[idx, 'location_state'] = fill_state

In [ ]:
# location_country 열의 최빈값을 계산
most_frequent_country = users_['location_country'].mode()[0]
# NaN 값을 최빈값으로 대체
users_['location_country'] = users_['location_country'].fillna(most_frequent_country)

In [ ]:
# user,book에서 사용할 열 정의
users_final = users_[['user_id', 'age', 'age_range', 'location_country']]
books_final = books_[['isbn', 'book_title', 'book_author', 'publisher', 'language', 'high_category', 'publication_range']]

## Model

In [ ]:
# 데이터 병합
train = train.merge(users_final, on='user_id').merge(books_final, on='isbn')
test = test.merge(users_final, on='user_id').merge(books_final, on='isbn')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# user_id 별 review_counts 계산
user_id_counts = train['user_id'].value_counts()
train['user_review_counts'] = train['user_id'].map(user_id_counts)
test['user_review_counts'] = test['user_id'].map(user_id_counts)
test['user_review_counts'] = test['user_review_counts'].fillna(0)

In [ ]:
# isbn 별 review_counts 계산
book_isbn_counts = train['isbn'].value_counts()
train['book_review_counts'] = train['isbn'].map(book_isbn_counts)
test['book_review_counts'] = test['isbn'].map(book_isbn_counts)
test['book_review_counts'] = test['book_review_counts'].fillna(0)

#### 파생변수: 유저별 평균 평점(`average_rating`)

##### Steam Rating Formula

$$ 
\begin{aligned} 
  \text{average rating} =& \frac{\text{num of positive}}{\text{num of review}} \\ 
  \text{score} =& \text{average rating} - (\text{average rating} - 5.5) \cdot 2^{-\log_{10}^{\text{num of reviews}}}
\end{aligned} 
$$

##### 베이지안 평균(Bayesian average)

$$ \text{weighted rating} = \frac{v}{v+m} \cdot R + \frac{m}{v+m} \cdot C $$

- $R$: 유저의 실제 평균 평점

- $v$: 해당 유저가 남긴 평점 수

- $m$: 평점 신뢰도를 보정하기 위한 임계값 (예: 5나 10처럼 설정)

- $C$: 전체 데이터의 평균 평점

In [ ]:
# Steam Rating Formula 함수 정의
def steam_rating_formula(ratings):
    num_reviews = len(ratings)
    # 평점의 평균 계산
    avg_rating = ratings.mean() if len(ratings) > 0 else np.nan  # rating이 없는 경우 NaN 반환
    
    # Steam Rating Formula에 따라 점수 계산
    score = avg_rating - (avg_rating - 5.5) * 2 ** (-np.log10(num_reviews + 1))
    
    return score


# 베이지안 평균 함수 정의
def bayesian_average(ratings, m=10):
    global_avg = train['rating'].mean()  # 전체 평점의 평균
    num_ratings = len(ratings)  # 유저가 남긴 평점 수
    if num_ratings == 0:
        return global_avg  # 리뷰가 없는 경우 전체 평균 반환
    avg_rating = ratings.mean()  # 유저의 평균 평점
    bayesian_score = (num_ratings / (num_ratings + m)) * avg_rating + (m / (num_ratings + m)) * global_avg
    
    return bayesian_score

In [ ]:
# 특정 평점 수 이상 매긴 유저를 담은 데이터프레임 생성 & user_id 인덱스 저장 (현재는 1로 설정: 전체 데이터)
user_rating_df = train.groupby('user_id')['rating'].count()
user_rating_idx = user_rating_df[user_rating_df >= 1].index

# 특정 평점 수 이상 매긴 유저 정보만 담은 데이터프레임 생성
heavy_user_df = train[train['user_id'].isin(user_rating_idx)]

# 유저별 평점 분포 확인 (유저별 rating 값의 빈도수 계산) -> 매긴 적 없는 평점 level에는 0으로 대체
rating_distribution = heavy_user_df.groupby('user_id')['rating'].value_counts().unstack(fill_value=0)

# 유저별 평점 분포 및 평균 계산
heavy_user_averages = heavy_user_df.groupby('user_id')['rating'].agg(
    num_rating=lambda x: x.count(),
    arithmetic_mean=np.mean,
    harmonic_mean=lambda x: hmean(x) if (x > 0).all() else np.mean,  # 조화평균은 0이 아닌 값만 포함해야 한다. (역수를 취해야 하므로)
    steam_rating=lambda x: steam_rating_formula(x),
    bayesian_mean=lambda x: bayesian_average(x, m=10)
)

# 결과를 유저별 평점 분포와 함께 결합
result_df = pd.concat((rating_distribution, heavy_user_averages), axis=1)

In [ ]:
def match_average_ratings(merged_df: pd.DataFrame, calculated_df: pd.DataFrame) -> pd.DataFrame:
    # 기존 데이터프레임에 'user_id'를 기준으로 평균 점수들 병합
    merged_df = merged_df.merge(calculated_df[['num_rating', 'arithmetic_mean', 'harmonic_mean', 'steam_rating', 'bayesian_mean']],
                                on='user_id', how='left')
    
    # 'average_rating' 계산
    # 먼저 'num_rating' 기준으로 조건을 처리할 수 있는 변수 생성
    condition_0 = merged_df['num_rating'] == 0
    condition_1_10 = (merged_df['num_rating'] > 0) & (merged_df['num_rating'] < 10)
    condition_10_20 = (merged_df['num_rating'] >= 10) & (merged_df['num_rating'] < 20)
    condition_above_20 = merged_df['num_rating'] >= 20
    
    # 각 조건에 맞는 평균값 할당
    merged_df['average_rating'] = 5.5  # 기본값을 5.5로 설정 (예시로)
    # merged_df.loc[condition_0, 'average_rating'] = merged_df['bayesian_mean']
    merged_df.loc[condition_1_10, 'average_rating'] = merged_df['steam_rating']
    merged_df.loc[condition_10_20, 'average_rating'] = merged_df['bayesian_mean']
    merged_df.loc[condition_above_20, 'average_rating'] = merged_df['harmonic_mean']

    return merged_df

In [ ]:
train = match_average_ratings(train, result_df)

In [ ]:
# 1. train과 test의 user_id를 집합(set) 형태로 저장하여 필요한 user_id 추출
train_users = set(train['user_id'].unique())
test_users = set(test['user_id'].unique())

# test에만 존재하는 user_id (처음 등장하는 유저)
new_users_in_test = test_users - train_users
common_users_in_test = test_users & train_users  # 교집합으로 추출

# 2. 처음 등장하는 유저에 대해 데모그래픽 정보를 기반으로 average_rating 계산
def calculate_demographic_average(train_df, threshold=10):
    # (age, location_country)별로 평균 평점 계산
    demographic_avgs = train_df.groupby(['age', 'location_country']).apply(
        lambda group: pd.Series({
            'average_rating': group['steam_rating'].mean() if group['num_rating'].mean() < threshold else group['harmonic_mean'].mean()
        })
    ).reset_index()
    return demographic_avgs

# demographic 평균 데이터프레임 생성
demographic_avgs = calculate_demographic_average(train, threshold=10)

# cold_start_users에는 demographic 평균을 기준으로 average_rating을 추가
cold_start_users = test[test['user_id'].isin(new_users_in_test)].copy()
cold_start_users = cold_start_users.merge(demographic_avgs, on=['age', 'location_country'], how='left')
cold_start_users['average_rating'].fillna(5.5, inplace=True)

# 3. common_users는 user_id를 기준으로 tmp_train에서 average_rating 값을 매핑
average_rating_map = train.set_index('user_id')['average_rating'].to_dict()
test['average_rating'] = test['user_id'].map(average_rating_map)

# 4. 처음 등장하는 유저에 대해 계산한 average_rating 값으로 업데이트
cold_start_map = cold_start_users.set_index('user_id')['average_rating'].to_dict()
test.loc[test['user_id'].isin(new_users_in_test), 'average_rating'] = test['user_id'].map(cold_start_map)

#### 피처 선택

In [ ]:
cat_col = ['isbn', 'book_title', 'book_author', 'publisher', 'language', 'high_category', 'publication_range', 'user_id', 'age_range', 'location_country']
num_col = ['rating', 'average_rating', 'user_review_counts', 'book_review_counts']

for df in [train, test] :
    for cat in cat_col :
        df[cat] = df[cat].astype('str')
    for num in num_col :
        df[num] = df[num].astype('float')

In [ ]:
for col in cat_col:
    combined_values = pd.concat([train[col], test[col]]).unique()
    train[col] = pd.Categorical(train[col], categories=combined_values).codes
    test[col] = pd.Categorical(test[col], categories=combined_values).codes

In [ ]:
# METRIC 함수
def calculate_metrics(y_true, y_pred):
    metrics = {
        'RMSE' : root_mean_squared_error(y_true, y_pred),
        'MSE' : mean_squared_error(y_true, y_pred),
        'MAE' : mean_absolute_error(y_true, y_pred)
    }
    return metrics

In [ ]:
# Stratified k-fold
def skf_train(X_data, y_data, params):
    skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    valid_rmse = []
    valid_mse = []
    valid_mae = []
    pred_df = pd.DataFrame()

    for fold, (train_idx, valid_idx) in tqdm.tqdm(enumerate(skf.split(X_data, y_data)), total = skf.n_splits) : 
        
        # Train Set과 Valid Set 분할    
        X_train, y_train = X_data.iloc[train_idx], y_data.iloc[train_idx]
        X_valid, y_valid = X_data.iloc[valid_idx], y_data.iloc[valid_idx]
        
        train_data = Pool(data = X_train, label = y_train, cat_features = cat_col)
        valid_data = Pool(data = X_valid, label = y_valid, cat_features = cat_col)
        
        cat_model = CatBoostRegressor(**params, iterations = 1000, 
                                    loss_function = 'RMSE', eval_metric = 'RMSE', 
                                    use_best_model = True, random_state = 42,
                                    cat_features = [i for i in range(0, 10)])
        cat_model.fit(train_data, eval_set = [train_data, valid_data], use_best_model = True, verbose = 100, early_stopping_rounds = 100)
        
        # 모델 RMSE
        valid_metrics = calculate_metrics(y_valid, cat_model.predict(X_valid))
        print(f"Fold {fold + 1} Valid RMSE: {valid_metrics['RMSE']}")
        print(f"Fold {fold + 1} Valid MSE:  {valid_metrics['MSE']}")
        print(f"Fold {fold + 1} Valid MAE:  {valid_metrics['MAE']}")
        valid_rmse.append(valid_metrics['RMSE'])
        valid_mse.append(valid_metrics['MSE'])
        valid_mae.append(valid_metrics['MAE'])

        wandb.log({
            'Valid RMSE': valid_metrics['RMSE'],
            'Valid MSE': valid_metrics['MSE'],
            'Valid MAE': valid_metrics['MAE']
        })
        
        # Predict
        pred = cat_model.predict(test.drop(['rating'], axis = 1))
        pred_df[f'pred_{fold}'] = pred
        
    print(f'RMSE 평균 : {np.array(valid_rmse).mean():.4f} \n')

    params = json.dumps(params)
    wandb.log({
        'Valid RMSE': np.array(valid_rmse).mean(),
        'Valid MSE': np.array(valid_mse).mean(),
        'Valid MAE': np.array(valid_mae).mean(),
        'param': params,
        'features': list(X_data.columns)
    })
    wandb.finish()

    return pred_df

In [ ]:
# Stratified k-fold Optuna
def optuna_train(X_data, y_data):
    def train(X_data, y_data, params):
        
        # Train Set과 Valid Set 분할    
        X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)
        
        train_data = Pool(data = X_train, label = y_train, cat_features = cat_col)
        valid_data = Pool(data = X_valid, label = y_valid, cat_features = cat_col)
        
        cat_model = CatBoostRegressor(**params, iterations = 500, 
                                    loss_function = 'RMSE', eval_metric = 'RMSE', 
                                    use_best_model = True, random_state = 42,
                                    cat_features = [i for i in range(0, 10)])
        cat_model.fit(train_data, eval_set = [train_data, valid_data], use_best_model = True,
                    verbose = 500, early_stopping_rounds = 100)
        
        valid_metrics = calculate_metrics(y_valid, cat_model.predict(X_valid))

        return valid_metrics['RMSE']

    def objective(trial):
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'depth': trial.suggest_int('depth', 3, 10),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
            'boosting_type': 'Plain',
            'bootstrap_type': 'MVS',
            'devices': 'cuda',
        }
        return train(X_data, y_data, params=params)
        
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='minimize', sampler=sampler)
    study.optimize(objective, n_trials=50)
    return study.best_params

In [ ]:
X_data, y_data = train.drop(
    columns = ['rating', 'age', 'num_rating', 'arithmetic_mean', 'harmonic_mean', 'steam_rating','bayesian_mean']
), train['rating']
test = test.drop(columns='age')

if opt:
    best_params = optuna_train(X_data, y_data)
else:
    best_params = {
        'learning_rate': 0.1895759434037735, 
        'depth': 8, 
        'l2_leaf_reg': 4, 
        'colsample_bylevel': 0.6758183738140613,
        'boosting_type': 'Plain',
        'bootstrap_type': 'MVS',
        'devices': 'cuda',
    }
pred_df = skf_train(X_data, y_data, params=best_params)

## Submission

In [ ]:
sub

In [ ]:
sub['rating'] = (pred_df['pred_0'] + pred_df['pred_1'] + pred_df['pred_2'] + pred_df['pred_3'] + pred_df['pred_4'] + 
                               pred_df['pred_5'] + pred_df['pred_6'] + pred_df['pred_7'] + pred_df['pred_8'] + pred_df['pred_9']) / 10
submit = sub[['user_id', 'isbn', 'rating']]
submit['rating'] = submit['rating'].clip(1, 10)  # 1~10을 벗어나는 값 clipping
submit

In [ ]:
submit.to_csv('submit.csv', index = False)

In [ ]:
submit['rating'].describe()

count    76699.000000
mean         7.133368
std          1.511893
min          1.000000
25%          6.337392
50%          7.318706
75%          8.186606
max         10.000000
Name: rating, dtype: float64